# Exploration of the experiments

Analyses we care about:
* Plots for all the different experiments


* proportion concluded with a deal
* final payoff per side
* how does starting affect results


In [161]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [162]:
import os 
import glob

import pandas as pd 
import numpy as np
import hydra
from omegaconf import DictConfig, OmegaConf
import yaml
from dataclasses import dataclass, field

import matplotlib.pyplot as plt
from hydra import initialize, compose

from omegaconf import OmegaConf, open_dict
from hydra.core.global_hydra import GlobalHydra
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from src.utils import unpack_nested_yaml, load_hydra_config

import omegaconf
from typing import List

## Move this class to a better location alter. 
For now it lets us open a config, import the negotiations, and begin looking at results.

Analysis we care about


Group by based on:
* message history
* note size
* message size
* game type
* 

In [219]:
game_mapping = {
    "rp_family_employees": "distributive", 
    "rp_financing":"distributive", 
    "rp_contingent_liability": "compatible", 
    "rp_non_compete_period": "distributive"
}

In [350]:
@dataclass
class NegotiationHistory:
    """
    Load and filter a completed negotiation run.
    
    :param history_path:
    """
    history_path: str
    meets_criteria: bool = False
    run_filters: dict = field(default_factory=list)
    debug: bool = True
        
    def __post_init__(self):
        try:
            self.cfg = load_hydra_config(os.path.join("..", self.history_path, ".hydra/"))
        except:
            self.cfg = None
            
        try:
            self.df = pd.read_csv(os.path.join(self.history_path, "processed_negotiation.csv"))
        except FileNotFoundError:
            if self.debug:
                print(f"Couldn't find file '{os.path.join(self.history_path, 'processed_negotiation.csv')}'")
            else:
                pass

        if (len(self.run_filters)>0) and (self.cfg is not None):
            self.meets_criteria = self.search_list(self.run_filters)
        
        if (self.meets_criteria) and (self.cfg is not None) and hasattr(self, "df"):
            # load in some of the key data
            self.agent_1 = self.cfg["experiments"]["agent_1"]
            self.agent_2 = self.cfg["experiments"]["agent_2"]
            self.agent_start_id = self.cfg["experiments"]["negotiation_protocol"]["start_agent_index"]
            self.number_rounds = max(self.df['round'])
            self.completion_reason = self.df.tail(1)["completion_reason"].values[0]
            self.issues = self.cfg["experiments"]["game"]["issues"]
            self.num_issues = len(self.issues)
            self.issue_weights = self.cfg['experiments']['game']['issue_weights']
            self.game_type = self.issues_to_types(self.issue_weights, self.issues)

            self.last_row_1 = self.df[self.df["agent_id"]==0].tail(1)
            self.last_row_2 = self.df[self.df["agent_id"]==1].tail(1)
            self.agent_1_mean_msg_length, self.agent_2_mean_msg_length = self.last_row_1["msg_length"].mean(), self.last_row_2["msg_length"].mean()
            self.agent_1_mean_note_length, self.agent_2_mean_note_length = self.last_row_1["note_length"].mean(), self.last_row_2["note_length"].mean()
            self.agent_1_normalized_payoff, self.agent_2_normalized_payoff = self.last_row_1["normalized_payoff"].mean(), self.last_row_2["normalized_payoff"].mean()
            # self.agent_1_normalized_issue_payoff, self.agent_2_normalized_issue_payoff = self.last_row_1["normalized_issue_payoff"].mean(), self.last_row_2["normalized_issue_payoff"].mean()


            
    def return_list(self):
        col_names = [
            "agent_1_msg_max_len", "agent_2_msg_max_len",
            "agent_1_note_max_len", "agent_2_note_max_len",
            "agent_1_msg_input_note_history", "agent_2_msg_input_note_history",
            "agent_1_msg_input_msg_history", "agent_2_msg_input_msg_history",
            "agent_1_note_input_note_history", "agent_2_note_input_note_history",
            "agent_1_note_input_msg_history", "agent_2_note_input_msg_history",
            "agent_1_model_name", "agent_2_model_name",
            "agent_1_mean_msg_length", "agent_2_mean_msg_length",
            "agent_1_mean_note_length", "agent_2_mean_note_length",
            "agent_1_normalized_payoff", "agent_2_normalized_payoff",
#             "agent_1_normalized_issue_payoff", "agent_2_normalized_issue_payoff",
            "agent_start_id",
            "game_type",
            "number_rounds",
        ]
        cols = [
             self.agent_1.msg_max_len, self.agent_2.msg_max_len,
             self.agent_1.note_max_len, self.agent_2.note_max_len,
             self.agent_1.msg_input_note_history, self.agent_2.msg_input_note_history, 
             self.agent_1.msg_input_msg_history, self.agent_2.msg_input_msg_history, 
             self.agent_1.note_input_note_history, self.agent_2.note_input_note_history,
             self.agent_1.note_input_msg_history, self.agent_2.note_input_msg_history,
             self.agent_1.model_name, self.agent_2.model_name, 
             self.agent_1_mean_msg_length, self.agent_2_mean_msg_length,
             self.agent_1_mean_note_length, self.agent_2_mean_note_length,
             self.agent_1_normalized_payoff, self.agent_2_normalized_payoff,
#              self.agent_1_normalized_issue_payoff, self.agent_2_normalized_issue_payoff,
             self.agent_start_id, 
             self.game_type, 
             self.number_rounds, 
            
         ]
        return col_names, cols

    def search_list(self, run_restriction: List[dict]):
        config = self.cfg["experiments"]
        contains_restrictions = []
        for key_val in run_restriction:
            for key, val in key_val.items():
                if key[0] == "~":
                    key = key[1:]
                    contains_restrictions.append(not search_config(config, key, val))
                else:
                    contains_restrictions.append(search_config(config, key, val))
        if all(contains_restrictions):
            return True
        return False
            
    @staticmethod
    def search_config(config, key, val, accu=0):
        def search(config, key, val, accu):
            if hasattr(config, "items"):
                for k, v in config.items():
                    if (k==key) & (v==val):
                        accu += 1
                    elif isinstance(v, omegaconf.dictconfig.DictConfig):
                        accu += search(v, key, val, accu)
                    elif isinstance(v, list):
                        for d in v:
                            accu += search(v, key, val, accu)
            return accu
        accu = search(config, key, val, 0)
        if accu > 0:
            return True
        return False
    
    @staticmethod
    def issues_to_types(issue_weights, issue_names):
        integrative = "non-integrative"
        compatible = ""
        if "rp_contingent_liability" in issue_names:
            compatible = "compatible"
        if issue_weights[0] != issue_weights[1]:
            integrative = "integrative"
        return integrative + " " + compatible
        
        
    def create_name(self):
        return f"""Agent {self.agent_start_id} starts negotiations
Agent 0: {self.agent_1.external_description.name} vs. Agent 1: {self.agent_2.external_description.name})
{self.agent_1.msg_max_len} (max msg len) | {self.agent_2.msg_max_len} (max msg len)
{self.agent_1.note_max_len} (max note len) | {self.agent_2.note_max_len} (max note len)
{self.agent_1.model_name} | {self.agent_2.model_name}
"""

In [351]:
def extract_dictionary(x):
    try: 
        return eval(x)
    except:
        return x

In [390]:
runs = glob.glob("logs/inference/runs/*")

In [391]:
runs = [k for k in runs if k >= "logs/inference/runs/2023-08-21_13-35"]

In [354]:
(history.meets_criteria) and (history.cfg is not None) and hasattr(history, "df")

True

In [392]:
runs

['logs/inference/runs/2023-08-21_15-22-47',
 'logs/inference/runs/2023-08-21_15-38-26',
 'logs/inference/runs/2023-08-21_15-29-36',
 'logs/inference/runs/2023-08-21_13-49-05',
 'logs/inference/runs/2023-08-21_14-52-54',
 'logs/inference/runs/2023-08-21_17-27-56',
 'logs/inference/runs/2023-08-21_14-23-08',
 'logs/inference/runs/2023-08-21_17-41-50',
 'logs/inference/runs/2023-08-21_16-13-52',
 'logs/inference/runs/2023-08-21_13-58-29',
 'logs/inference/runs/2023-08-21_17-26-38',
 'logs/inference/runs/2023-08-21_16-04-16',
 'logs/inference/runs/2023-08-21_17-30-52',
 'logs/inference/runs/2023-08-21_16-01-23',
 'logs/inference/runs/2023-08-21_14-38-42',
 'logs/inference/runs/2023-08-21_16-00-18',
 'logs/inference/runs/2023-08-21_16-15-47',
 'logs/inference/runs/2023-08-21_14-49-42',
 'logs/inference/runs/2023-08-21_14-16-19',
 'logs/inference/runs/2023-08-21_13-52-19',
 'logs/inference/runs/2023-08-21_14-08-16',
 'logs/inference/runs/2023-08-21_14-46-18',
 'logs/inference/runs/2023-08-21

In [381]:
vals = []
restrictions = [
    {'external_description': {'name': 'Representative'}, 
     "temperature": 0, 
     "~model_name": "claude-2"
    }
]
for i, run in enumerate(runs):
    history = NegotiationHistory(run, restrictions)
    if not history.meets_criteria:
        continue
    if hasattr(history, "df"):
        print(i)
        col_names = history.return_list()[0]
        vals.append(history.return_list()[1])

0
Couldn't find file 'logs/inference/runs/2023-08-21_15-38-26/processed_negotiation.csv'
2
3
4
5
Couldn't find file 'logs/inference/runs/2023-08-21_14-23-08/processed_negotiation.csv'
7
8
9
10
11
12
13
Couldn't find file 'logs/inference/runs/2023-08-21_14-38-42/processed_negotiation.csv'
Couldn't find file 'logs/inference/runs/2023-08-21_16-00-18/processed_negotiation.csv'
16
17
18
Couldn't find file 'logs/inference/runs/2023-08-21_13-52-19/processed_negotiation.csv'
20
21
Couldn't find file 'logs/inference/runs/2023-08-21_14-05-23/processed_negotiation.csv'
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
Couldn't find file 'logs/inference/runs/2023-08-21_16-00-16/processed_negotiation.csv'
39
Couldn't find file 'logs/inference/runs/2023-08-21_14-58-52/processed_negotiation.csv'
41
42
Couldn't find file 'logs/inference/runs/2023-08-21_14-19-08/processed_negotiation.csv'
Couldn't find file 'logs/inference/runs/2023-08-21_16-01-15/processed_negotiation.csv'
45
46
Couldn't find file 'logs/in

In [383]:
df = pd.DataFrame(vals, columns=col_names)

In [384]:
df.groupby(["agent_1_msg_max_len", "agent_1_note_max_len", "agent_2_msg_max_len", "agent_2_note_max_len", "agent_start_id"])[["agent_1_normalized_payoff", "agent_2_normalized_payoff"]].mean()

agent_1_normalized_payoff   
agent_1_msg_max_len agent_1_note_max_len agent_2_msg_max_len agent_2_note_max_len agent_start_id                              
32                  32                   32                  32                   0                                0.412854  \
                                         150                 150                  0                                0.465875   
                    150                  32                  150                  0                                0.511430   
150                 32                   32                  150                  0                                0.440954   
                                         150                 32                   0                                0.376192   
                    150                  32                  32                   0                                0.336305   
                                         150                 150                  0                                0.358333   

                                                                                                  agent_2_normalized_payoff  
agent_1_msg_max_len agent_1_note_max_len agent_2_msg_max_len agent_2_note_max_len agent_start_id                             
32                  32                   32                  32                   0                                0.632858  
                                         150                 150                  0                                0.650595  
                    150                  32                  150                  0                                0.551428  
150                 32                   32                  150                  0                                0.514284  
                                         150                 32                   0                                0.630358  
                    150                  32                  32                   0                                0.644643  
                                         150                 150                  0                                0.708333

In [218]:
log_of_histories = []
restrictions = [
    {'external_description': {'name': 'Representative'}, 
     "temperature": 0, 
     "max_rounds": 15, 
     "start_agent_index": 0,
     "~model_name": "claude-2"
    }
]
verbose = False

xyz = []

i = 0
for i, run in enumerate(runs):
    history = NegotiationHistory(run, restrictions)
    if not history.meets_criteria:
        continue
    if hasattr(history, "df"):
        
        if "completion_reason" in history.df.columns:
            payoffs = {0: 0, 1:0}
            print(f"Run: {i}")
            i+=1
            print(f"Number rounds: {max(history.df['round'])}")
            completion_reason = history.df.tail(1)["completion_reason"].values[0]
            print(f"Completion reason: {completion_reason}")
            num_issues = len(history.cfg["experiments"]["game"]["issues"])
            print(f"Number of issues: {num_issues}")
            print(f"Payoffs: {history.cfg['experiments']['game']['issue_weights']}")
            log_of_histories.append(history)
            
#             history.df["offers_in_message"] = history.df["offers_in_message"].apply(lambda x: extract_dictionary(x))
            
            agents = [history.agent_1, history.agent_2]
            sum_payoff = history.df.iloc[-2:]["normalized_payoff"].sum()
            for idx, row in history.df.iloc[-2:].iterrows():
                agent = agents[row['agent_id']]
                payoffs[row["agent_id"]] += row["normalized_payoff"]
                print(f"Agent id: {row['agent_id']}")
                print(f"  msg_max_len: {agent['msg_max_len']}")
                print(f"  note_max_len: {agent['note_max_len']}")
                print(f"  Final payoff: {row['normalized_payoff']}")
                print(f"  Offer in notes: {row['issues_state']}")
#                 print(f"  Offer in message: {row['offers_in_message']}")
                
                if verbose:
                    print(f"Note: {row['note']}")
                    print(f"\nMessage: {row['message']}")

            if completion_reason == "issues agreed upon":
                xyz.append((agent['msg_max_len'], agent['note_max_len'], num_issues, sum_payoff, payoffs[0], payoffs[1]))
            print("============================================================")
#     groups = history.df.groupby("agent_id")
#     fig, ax = plt.subplots()

#     for name, group in groups:
#         x = group["round"]
#         y = group["normalized_payoff"]
        
#         ax.plot(x,y, label=name, alpha=0.5)
#     plt.title(run_name)
#     plt.legend()
#     plt.show()



Run: 0
Number rounds: 5
Completion reason: issues agreed upon
Number of issues: 2
Payoffs: [[1500, 1500], [1500, 1500]]
Agent id: 0
  msg_max_len: 32
  note_max_len: 32
  Final payoff: 0.55
  Offer in notes: {'non-compete period': '6 years', 'financing terms': '25 million now'}
Agent id: 1
  msg_max_len: 150
  note_max_len: 150
  Final payoff: 0.45
  Offer in notes: {'non-compete period': '6 years', 'financing terms': '25 million now'}
Run: 1
Number rounds: 6
Completion reason: issues agreed upon
Number of issues: 2
Payoffs: [[1500, 1500], [1500, 1500]]
Agent id: 1
  msg_max_len: 150
  note_max_len: 32
  Final payoff: 0.75
  Offer in notes: {'non-compete period': '2 years', 'financing terms': '27 million now'}
Agent id: 0
  msg_max_len: 150
  note_max_len: 32
  Final payoff: 0.25
  Offer in notes: {'non-compete period': '2 years', 'financing terms': '27 million now'}
Run: 3
Number rounds: 5
Completion reason: issues agreed upon
Number of issues: 2
Payoffs: [[2500, 1000], [1000, 2500]]


Run: 22
Number rounds: 3
Completion reason: issues agreed upon
Number of issues: 2
Payoffs: [[500, 2500], [1000, 1000]]
Agent id: 1
  msg_max_len: 32
  note_max_len: 150
  Final payoff: 0.6
  Offer in notes: {'contingent liability': '60 percent', 'family employees': '6 employees'}
Agent id: 0
  msg_max_len: 150
  note_max_len: 32
  Final payoff: 0.43333
  Offer in notes: {'contingent liability': '60 percent', 'family employees': '6 employees'}
Run: 23
Number rounds: 8
Completion reason: issues agreed upon
Number of issues: 2
Payoffs: [[1500, 1500], [1500, 1500]]
Agent id: 1
  msg_max_len: 32
  note_max_len: 32
  Final payoff: 0.7
  Offer in notes: {'non-compete period': '2 years', 'financing terms': '26 million now'}
Agent id: 0
  msg_max_len: 32
  note_max_len: 32
  Final payoff: 0.3
  Offer in notes: {'non-compete period': '2 years', 'financing terms': '26 million now'}
Run: 25
Number rounds: 15
Completion reason: max rounds reached
Number of issues: 2
Payoffs: [[500, 2500], [1000, 1

In [178]:
col_of_interest = "agent_2_payoff"

d = pd.DataFrame(xyz, columns=["msg","note","num_issues", "sum_payoff", "agent_1_payoff", "agent_2_payoff"])
d = d.dropna()
d.groupby("msg")[col_of_interest].mean()

msg
32     0.579464
150    0.545714
Name: agent_2_payoff, dtype: float64

In [179]:
d.groupby("note")[col_of_interest].mean()

note
32     0.611224
150    0.514285
Name: agent_2_payoff, dtype: float64

In [180]:
d.groupby(["note","msg"])[col_of_interest].mean()

note  msg
32    32     0.637500
      150    0.576190
150   32     0.521427
      150    0.500000
Name: agent_2_payoff, dtype: float64

In [181]:
d = d.dropna()

In [182]:
# d[d["note"]==150].sort_values(by="msg")

In [183]:
d.groupby(["num_issues"])[col_of_interest].mean()

num_issues
2    0.566483
Name: agent_2_payoff, dtype: float64

In [184]:
d.groupby(["num_issues","msg","note"])[col_of_interest].mean()

num_issues  msg  note
2           32   32      0.637500
                 150     0.521427
            150  32      0.576190
                 150     0.500000
Name: agent_2_payoff, dtype: float64

In [136]:
d

,msg,note,num_issues,sum_payoff,agent_1_payoff,agent_2_payoff
0,150,150,1,1.000000,0.400000,0.600000
1,32,32,1,1.000000,0.200000,0.800000
2,150,32,3,0.834211,0.278947,0.555263
3,32,150,3,1.000000,0.576316,0.423684
4,32,32,3,1.055263,0.289474,0.765789
5,150,32,1,1.000000,0.200000,0.800000
6,32,150,3,1.000000,0.400000,0.600000
8,150,150,3,1.055263,0.889474,0.165789
9,32,32,3,1.000000,0.523684,0.476316
10,150,150,1,1.000000,0.200000,0.800000
